<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=addec3f2b98df4c434444626e0dd83a2ad8977a859be4715d04c2e7265cd36fc
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-26 11:10:58
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.37 C
-------------------
Today PnL: -1.39 L (-1.01%)
Current PnL: -22.45 L (-15.1%)
CY Booked + Current PnL: -11.01 L (-7.41%)
-------------------
Total profit:  92.25 K
Total loss:  -23.37 L
-------------------
Total Booked + Current PnL: 15.97 L (13.12%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.34%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 88.32 L (64.4%)
Deployed:  1.22 C
Current:  1.37 C
CAGR/XIRR %: 7.25%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,10.86,51.0,H-MC,3.86,191250.0,26580.0,16754.0,-0.77,16.14,8.76,26.31,89.0,1.59,1.39,25.41,X40N,NTT,AC
77,TTKPRESTIG,770.00,95.09,38.0,M-SC,2.78,84063.0,-16714.0,16804.0,-1.76,-16.59,19.99,0.09,245.0,-0.99,0.61,10.44,OX40N,NTT,DURABLES
50,NATIONALUM,244.55,-46.15,48.0,H-MC,10.90,106795.0,6291.0,22811.0,-2.00,6.26,21.36,28.96,79.0,0.28,0.78,42.35,MH,ATH,METALS
42,ITC,452.00,-39.19,42.0,H-LC,2.18,196822.0,-3316.0,24209.0,0.60,-1.66,12.30,10.44,4.0,-0.14,1.44,3.81,X40,NTT,FMCG
57,RELIANCE,1533.00,-14.41,43.0,H-LC,4.07,214516.0,4150.0,24626.0,0.20,1.97,11.48,13.68,37.0,0.17,1.56,18.82,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TATAMOTORS,1065.00,-22.16,44.0,M-LC,2.36,246441.0,-30082.0,140151.0,2.20,-10.88,56.87,39.81,54.0,-0.21,1.80,18.11,XY24,NTT,AUTO
61,SAMMAANCAP,326.00,-171.59,58.0,M-SC,5.90,82878.0,-19332.0,112722.0,1.26,-18.91,136.01,91.37,208.0,-0.17,0.60,34.54,XY25,NTT,FINANCE
34,ICICIGI,2260.25,-18.25,53.0,H-MC,1.87,140541.0,4408.0,26717.0,0.92,3.24,19.01,22.86,91.0,0.16,1.03,16.26,X40,ATH,INSURANCE
42,ITC,452.00,-39.19,42.0,H-LC,2.18,196822.0,-3316.0,24209.0,0.60,-1.66,12.30,10.44,4.0,-0.14,1.44,3.81,X40,NTT,FMCG
64,SHALBY,327.00,1182.30,66.0,M-SC,12.74,174296.0,-6847.0,52306.0,0.21,-3.78,30.01,25.10,235.0,-0.13,1.27,39.68,XY24,NTT,HEALTHCARE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,HAPPSTMNDS,1488.71,-20.51,27.0,H-SC,10.52,85872.0,-42293.0,152320.0,-2.96,-33.00,177.38,85.85,132.0,-0.28,0.63,0.00,AR,ATH,IT
46,KPIGREEN,731.05,-3.26,31.0,H-SC,8.76,115189.0,-10108.0,69033.0,-2.86,-8.07,59.93,47.03,141.0,-0.15,0.84,44.23,MH,ATH,POWER
32,HINDZINC,730.22,21.48,54.0,M-LC,2.79,197664.0,-7412.0,119982.0,-2.65,-3.61,60.70,54.89,52.0,-0.06,1.44,19.61,X5K,ATH,METALS
60,SAIL,228.00,45.15,57.0,M-MC,10.26,229370.0,4408.0,163243.0,-2.43,1.96,71.17,74.53,192.0,0.03,1.67,34.85,XY24,BTT,STEEL
54,RAJESHEXPO,518.00,1691.67,41.0,L-SC,2.60,48879.0,-88298.0,88388.0,-2.42,-64.37,180.83,0.07,267.0,-1.00,0.36,21.56,OX40N,NTT,JEWELLERY


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,PGHH,17973.08,-30.02,52.0,H-MC,3.4,202530.0,1710.0,67058.0,-2.06,0.85,33.11,34.25,80.0,0.03,1.48,6.09,X40,ATH,FMCG


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,92.53,49.0,H-SC,10.19,121311.0,-21168.0,40688.0,-0.09,-14.86,33.54,13.70,163.0,-0.52,0.88,43.18,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,95.09,38.0,M-SC,2.78,84063.0,-16714.0,16804.0,-1.76,-16.59,19.99,0.09,245.0,-0.99,0.61,10.44,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-68.16,48.0,H-SC,1.40,219474.0,-50193.0,86517.0,-0.56,-18.61,39.42,13.47,138.0,-0.58,1.60,11.01,XY24,NTT,PAINTS
18,CERA,9475.0,-22.91,33.0,H-SC,1.86,140496.0,-35407.0,77427.0,-0.32,-20.13,55.11,23.89,149.0,-0.46,1.02,21.34,OX40N,NTT,CERAMICS
66,SIS,528.0,2080.83,40.0,H-SC,4.23,88013.0,-23019.0,46629.0,-0.27,-20.73,52.98,21.26,156.0,-0.49,0.64,18.91,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,45.15,57.0,M-MC,10.26,229370.0,4408.0,163243.0,-2.43,1.96,71.17,74.53,192.0,0.03,1.67,34.85,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,10.86,51.0,H-MC,3.86,191250.0,26580.0,16754.0,-0.77,16.14,8.76,26.31,89.0,1.59,1.39,25.41,X40N,NTT,AC
84,VOLTAS,1530.00,-3.18,41.0,H-MC,3.04,203265.0,11523.0,26242.0,-1.26,6.01,12.91,19.69,99.0,0.44,1.48,13.22,XY25,NTT,AC
17,CAMS,5211.76,-5.61,44.0,H-SC,1.76,107201.0,5197.0,38732.0,-1.03,5.09,36.13,43.06,122.0,0.13,0.78,23.08,X40N,ATH,MISC
79,UNIONBANK,163.00,-14.73,53.0,M-LC,6.83,154859.0,14019.0,30476.0,-1.85,9.95,19.68,31.59,66.0,0.46,1.13,39.08,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-46.15,48.0,H-MC,10.90,106795.0,6291.0,22811.0,-2.00,6.26,21.36,28.96,79.0,0.28,0.78,42.35,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-44.58,33.0,M-SC,1.50,93772.0,2274.0,76480.0,0.10,2.48,81.56,86.07,223.0,0.03,0.68,36.17,XR,NTT,DURABLES
11,BANKINDIA,190.00,-29.19,56.0,H-MC,8.23,182253.0,2445.0,109971.0,-1.05,1.36,60.34,62.52,88.0,0.02,1.33,35.91,XR,NTT,BANKS
37,INDIAMART,4850.92,-55.20,25.0,H-SC,10.24,125297.0,1961.0,131800.0,-0.40,1.59,105.19,108.45,119.0,0.01,0.91,24.30,AR,ATH,MISC
86,WIPRO,420.00,-16.52,33.0,M-LC,5.64,146971.0,-3974.0,113432.0,-2.05,-2.63,77.18,72.51,53.0,-0.04,1.07,3.23,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-17.51,30.0,H-LC,7.05,208076.0,-43692.0,96901.0,-1.64,-17.35,46.57,21.13,27.0,-0.45,1.52,11.92,X40,ATH,PAINTS
44,JIOFIN,387.00,-5.28,30.0,H-LC,12.97,257312.0,-10053.0,75495.0,-0.80,-3.76,29.34,24.48,48.0,-0.13,1.88,49.15,XY24,BTT,FINANCE
74,TCS,4476.75,-30.93,30.0,H-LC,12.07,274734.0,-71222.0,146076.0,-1.17,-20.59,53.17,21.64,1.0,-0.49,2.00,0.00,X40,ATH,IT
83,VBL,671.64,-20.19,31.0,H-LC,8.17,287228.0,-28614.0,141287.0,-0.37,-9.06,49.19,35.67,5.0,-0.20,2.10,3.54,X40N,ATH,FMCG
30,HCLTECH,1943.91,-3.23,36.0,H-LC,9.92,220522.0,-21402.0,84680.0,-1.35,-8.85,38.40,26.15,8.0,-0.25,1.61,4.21,X40,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-7.23,41.0,H-LC,1.68,229770.0,-16381.0,95584.0,-2.07,-6.65,41.60,32.18,16.0,-0.17,1.68,26.99,X200,ATH,IT
42,ITC,452.0,-39.19,42.0,H-LC,2.18,196822.0,-3316.0,24209.0,0.60,-1.66,12.30,10.44,4.0,-0.14,1.44,3.81,X40,NTT,FMCG
65,SIEMENS,4671.5,-2.83,42.0,H-LC,2.62,156050.0,-30045.0,77526.0,-1.12,-16.14,49.68,25.51,15.0,-0.39,1.14,15.00,AR,ATH,ELECTRICAL
57,RELIANCE,1533.0,-14.41,43.0,H-LC,4.07,214516.0,4150.0,24626.0,0.20,1.97,11.48,13.68,37.0,0.17,1.56,18.82,XY25,NTT,REFINERIES
51,NESTLEIND,1377.0,-11.41,43.0,H-LC,4.34,272563.0,7137.0,49661.0,-0.74,2.69,18.22,21.40,11.0,0.14,1.99,10.22,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-30.93,30.0,H-LC,12.07,274734.0,-71222.0,146076.0,-1.17,-20.59,53.17,21.64,1.0,-0.49,2.00,0.00,X40,ATH,IT
80,UNITDSPR,1644.00,-14.23,44.0,H-LC,7.16,224804.0,-9340.0,57954.0,-0.80,-3.99,25.78,20.77,86.0,-0.16,1.64,2.65,X40N,NTT,BREWERIES
83,VBL,671.64,-20.19,31.0,H-LC,8.17,287228.0,-28614.0,141287.0,-0.37,-9.06,49.19,35.67,5.0,-0.20,2.10,3.54,X40N,ATH,FMCG
42,ITC,452.00,-39.19,42.0,H-LC,2.18,196822.0,-3316.0,24209.0,0.60,-1.66,12.30,10.44,4.0,-0.14,1.44,3.81,X40,NTT,FMCG
30,HCLTECH,1943.91,-3.23,36.0,H-LC,9.92,220522.0,-21402.0,84680.0,-1.35,-8.85,38.40,26.15,8.0,-0.25,1.61,4.21,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-2.83,42.0,H-LC,2.62,156050.0,-30045.0,77526.0,-1.12,-16.14,49.68,25.51,15.0,-0.39,1.14,15.00,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.19,42.0,H-LC,2.18,196822.0,-3316.0,24209.0,0.60,-1.66,12.30,10.44,4.0,-0.14,1.44,3.81,X40,NTT,FMCG
19,CIPLA,1795.00,-23.21,36.0,H-LC,4.54,204651.0,151.0,41258.0,-1.05,0.07,20.16,20.25,10.0,0.00,1.49,8.81,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-17.51,30.0,H-LC,7.05,208076.0,-43692.0,96901.0,-1.64,-17.35,46.57,21.13,27.0,-0.45,1.52,11.92,X40,ATH,PAINTS
57,RELIANCE,1533.00,-14.41,43.0,H-LC,4.07,214516.0,4150.0,24626.0,0.20,1.97,11.48,13.68,37.0,0.17,1.56,18.82,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-30.93,30.0,H-LC,12.07,274734.0,-71222.0,146076.0,-1.17,-20.59,53.17,21.64,1.0,-0.49,2.00,0.00,X40,ATH,IT
40,INFY,2275.00,-19.96,42.0,H-LC,8.12,311832.0,-1056.0,172755.0,-1.40,-0.34,55.40,54.87,3.0,-0.01,2.27,6.29,X40,BTT,IT
42,ITC,452.00,-39.19,42.0,H-LC,2.18,196822.0,-3316.0,24209.0,0.60,-1.66,12.30,10.44,4.0,-0.14,1.44,3.81,X40,NTT,FMCG
83,VBL,671.64,-20.19,31.0,H-LC,8.17,287228.0,-28614.0,141287.0,-0.37,-9.06,49.19,35.67,5.0,-0.20,2.10,3.54,X40N,ATH,FMCG
1,ABB,7934.00,-40.74,42.0,H-LC,7.97,247565.0,-14054.0,133264.0,-0.83,-5.37,53.83,45.57,7.0,-0.11,1.81,5.34,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,44.0,L-SC,41.41,79680.0,-33869.0,73919.0,-2.05,-29.83,92.77,35.27,268.0,-0.46,0.58,95.02,XR,NTT,HOTELS
61,SAMMAANCAP,326.00,-171.59,58.0,M-SC,5.90,82878.0,-19332.0,112722.0,1.26,-18.91,136.01,91.37,208.0,-0.17,0.60,34.54,XY25,NTT,FINANCE
49,MASFIN,398.61,-19.60,40.0,H-SC,12.51,91545.0,-6435.0,28040.0,-1.18,-6.57,30.63,22.05,152.0,-0.23,0.67,33.07,XR,ATH,FINANCE
50,NATIONALUM,244.55,-46.15,48.0,H-MC,10.90,106795.0,6291.0,22811.0,-2.00,6.26,21.36,28.96,79.0,0.28,0.78,42.35,MH,ATH,METALS
78,UJJIVANSFB,60.00,92.53,49.0,H-SC,10.19,121311.0,-21168.0,40688.0,-0.09,-14.86,33.54,13.70,163.0,-0.52,0.88,43.18,OX40N,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1182.30,66.0,M-SC,12.74,174296.0,-6847.0,52306.0,0.21,-3.78,30.01,25.10,235.0,-0.13,1.27,39.68,XY24,NTT,HEALTHCARE
61,SAMMAANCAP,326.00,-171.59,58.0,M-SC,5.90,82878.0,-19332.0,112722.0,1.26,-18.91,136.01,91.37,208.0,-0.17,0.60,34.54,XY25,NTT,FINANCE
71,TANLA,1963.11,-18.69,58.0,H-SC,17.05,217658.0,-53103.0,383056.0,-1.30,-19.61,175.99,121.86,133.0,-0.14,1.59,70.27,AR,ATH,IT
81,VAIBHAVGBL,521.00,54.27,52.0,H-SC,5.61,133978.0,-48797.0,159863.0,-1.44,-26.70,119.32,60.77,125.0,-0.31,0.98,20.00,XR,NTT,JEWELLERY
7,ATULAUTO,844.00,3608.11,50.0,M-SC,7.14,115640.0,-31728.0,83538.0,-0.35,-21.53,72.24,35.16,236.0,-0.38,0.84,18.11,XY24,NTT,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.28
1,25,43.75
2,50,74.93


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.91
LC    32.45
MC    23.65
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.35
X40      14.77
X40N     11.41
XR       11.15
XY25     10.14
AR        8.34
OX40N     7.89
X200      1.68
MH        1.62
X5K       1.44
SR        1.22
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.39
H-LC    25.96
H-MC    20.70
M-SC    13.05
M-LC     5.44
M-MC     2.65
L-SC     1.47
L-LC     1.05
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.05,-6.70,41.91
IT,12.85,-19.31,84.36
FINANCE,10.08,-18.77,70.59
MISC,7.00,-19.80,86.40
BANKS,6.16,-18.09,80.21
PAINTS,5.66,-19.00,36.74
ELECTRICAL,5.48,-9.74,48.25
INSURANCE,3.78,-6.98,46.97
AC,3.58,1.51,14.98


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3221299.0,22
XR,1382014.0,14
AR,1190587.0,9
X40,825024.0,10
X40N,640535.0,9
OX40N,563201.0,10
XY25,447345.0,7
SR,254232.0,2
X5K,119982.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3766141.0,29
M-SC,1454340.0,17
H-MC,1269361.0,15
H-LC,1262186.0,15
M-LC,404041.0,4
M-MC,314013.0,2
L-SC,260197.0,3
L-MC,61345.0,1
L-LC,40023.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1253033.0      6
M-SC       XY24       832126.0      7
H-SC       AR         809577.0      5
           XR         801035.0      7
H-MC       XY24       576005.0      4
H-LC       X40        524621.0      5
H-SC       X40N       315390.0      4
M-MC       XY24       314013.0      2
H-SC       OX40N      263841.0      4
           SR         254232.0      2
H-LC       X40N       240499.0      3
H-MC       X40        224188.0      4
H-LC       AR         210790.0      2
H-MC       XY25       179403.0      2
L-SC       XR         171809.0      2
M-LC       XY24       170627.0      2
M-SC       AR         170220.0      2
           OX40N      138635.0      4
           XR         124422.0      2
M-LC       X5K        119982.0      1
H-LC       XY25       115197.0      3
M-LC       XR         113432.0      1
M-SC       XY25       112722.0      1
H-MC       XR         109971.0      1
H-LC       X200        95584.0      1
L-SC       OX40N       88388.0      1
H-MC       X40N        84646.0      2
M-SC       X40         76215.0      1
H-LC       XY24        75495.0      1
H-MC       OX40N       72337.0      1
H-SC       MH          69033.0      1
L-MC       XR          61345.0      1
L-LC       XY25        40023.0      1
H-MC       MH          22811.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
